In [ ]:
!git clone https://github.com/anminhhung/small_dog_cat_dataset

Cloning into 'small_dog_cat_dataset'...
remote: Enumerating objects: 2608, done.
remote: Total 2608 (delta 0), reused 0 (delta 0), pack-reused 2608 (from 1)
Receiving objects: 100% (2608/2608), 55.84 MiB | 19.31 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
import glob
import os
import cv2
import numpy as np
from skimage.feature import hog

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
import itertools
import matplotlib.pyplot as plt
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
   plt.imshow(cm, interpolation='nearest', cmap=cmap)
   plt.title(title)
   plt.colorbar()
   tick_marks = np.arange(len(classes))
   plt.xticks(tick_marks, classes, rotation=45)
   plt.yticks(tick_marks, classes)

   fmt = 'd'
   thresh = cm.max() / 2
   for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, format(cm[i, j], fmt),
               horizontalalignment="center",
               color="white" if cm[i, j] > thresh else "black")
   plt.tight_layout()
   plt.ylabel('True label')
   plt.xlabel('Predicted label')


Đọc file

In [ ]:
def read_file(path,target_size=(64,64)):
  datas = []
  labels = []

  for category in os.listdir(path):
    category_dir = os.path.join(path,category)
    for image_name in os.listdir(category_dir):
      image_path = os.path.join(category_dir,image_name)
      image = cv2.imread(image_path)
      image = cv2.resize(image,target_size)
      datas.append(image)
      labels.append(category)
  return np.array(datas),np.array(labels)

In [ ]:
train_dir = 'small_dog_cat_dataset/train'
test_dir = 'small_dog_cat_dataset/test'
target_size = (64,64)

train_datas,train_labels = read_file(train_dir,target_size)
test_datas,test_labels = read_file(test_dir,target_size)

Feature Extraction

In [ ]:
def hog_feature(data):
  hog_gray_features = []

  for image in data:
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    hog_features,_ = hog(gray_image, visualize=True,
                         block_norm= 'L2-Hys',
                         pixels_per_cell=(16, 16),
                         cells_per_block=(2, 2),
                         )
    hog_gray_features.append(hog_features)
  return np.array(hog_gray_features)

In [ ]:
train_data = hog_feature(train_datas)
test_data = hog_feature(test_datas)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(train_data,train_labels,test_size=0.4)

In [ ]:
clf = LogisticRegression()
clf.fit(X_train,y_train)
y_val_pred = clf.predict(X_val)
print('acc for val data: ', accuracy_score(y_val_pred,y_val))
y_pred = clf.predict(test_data)
print('acc for test data: ', accuracy_score(y_pred,test_labels))

acc for val data:  0.72375
acc for test data:  0.7183333333333334


In [ ]:
X_train,X_val,y_train,y_val = train_test_split(train_data,train_labels,test_size=0.3)
clf = LogisticRegression()
clf.fit(X_train,y_train)
y_val_pred = clf.predict(X_val)
print('acc for val data: ', accuracy_score(y_val_pred,y_val))
y_pred = clf.predict(test_data)
print('acc for test data: ', accuracy_score(y_pred,test_labels))

acc for val data:  0.725
acc for test data:  0.7183333333333334


In [ ]:
X_train,X_val,y_train,y_val = train_test_split(train_data,train_labels,test_size=0.2)
clf = LogisticRegression()
clf.fit(X_train,y_train)
y_val_pred = clf.predict(X_val)
print('acc for val data: ', accuracy_score(y_val_pred,y_val))
y_pred = clf.predict(test_data)
print('acc for test data: ', accuracy_score(y_pred,test_labels))

acc for val data:  0.69
acc for test data:  0.705


K-Fold validation

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

In [ ]:
best_model = None
best_test_acc = 0
for i, (train_index, val_index) in enumerate(kf.split(train_data)):
  X_train = []
  X_val = []
  y_train = []
  y_val = []

  for index in train_index:
    X_train.append(train_data[index])
    y_train.append(train_labels[index])

  for index in val_index:
    X_val.append(train_data[index])
    y_val.append(train_labels[index])

  clf = LogisticRegression()
  clf.fit(X_train,y_train)

  y_val_pred = clf.predict(X_val)
  val_acc = accuracy_score(y_val_pred,y_val)
  print('[Fold: {}] - acc for val data: {}'.format(i,val_acc))

  y_pred = clf.predict(test_data)
  test_acc = accuracy_score(y_pred,test_labels)
  print('[Fold: {}] - acc for test data: {}'.format(i,test_acc))
  print()

  if test_acc > best_test_acc:
    best_test_acc = test_acc
    best_model = clf


[Fold: 0] - acc for val data: 0.5425
[Fold: 0] - acc for test data: 0.6933333333333334

[Fold: 1] - acc for val data: 0.545
[Fold: 1] - acc for test data: 0.6933333333333334

[Fold: 2] - acc for val data: 0.725
[Fold: 2] - acc for test data: 0.7083333333333334

[Fold: 3] - acc for val data: 0.5625
[Fold: 3] - acc for test data: 0.7183333333333334

[Fold: 4] - acc for val data: 0.52
[Fold: 4] - acc for test data: 0.6983333333333334

